In [1]:
import datetime
from ib_async import *
import pandas as pd
from arcticdb import Arctic, QueryBuilder, LibraryOptions

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [2]:
ac = initialize_db('db')


In [4]:
ac.list_libraries()

['general', 'pnl', 'portfolio', 'spx500', 'stocks']

In [5]:
lib = ac.get_library('portfolio')
lib.list_symbols()

['U7706434',
 'test2',
 'DU3180649_AAPL',
 'U7706434_SMAG7',
 'DU3180649',
 'positions',
 'test3',
 'test']

In [7]:
lib.read('test3').data

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,...,close_dt,deleted,delete_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base,fx_rate
0,2024-07-24 11:03,AAPL,STK,37.0,3.647876,180.260541,223.979996,24.253481,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8287.260000,1617.620000,USD,0.0,DU3180649,7648.001772,1.083585
1,2024-07-24 11:03,CNR,STK,-699.0,-36.924325,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.260000,6118.460000,CAD,0.0,DU3180649,-77414.176421,1.494665
2,2024-07-24 11:03,SIE,STK,1.0,0.082425,181.640000,172.812302,-4.859997,,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,172.810000,-8.830000,EUR,0.0,DU3180649,172.810000,1.000000
3,2024-07-24 11:03,VOW,STK,-176.0,-9.306355,141.129400,110.860001,21.447976,,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-19511.360000,5327.410000,EUR,0.0,DU3180649,-19511.360000,1.000000
4,2024-07-24 11:05,AAPL,STK,37.0,3.647876,180.260541,223.979996,24.253481,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8287.259843,43.719455,USD,0.0,DU3180649,7648.001772,1.083585
5,2024-07-24 11:05,CNR,STK,-699.0,-36.924325,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.263623,-8.753157,CAD,0.0,DU3180649,-77414.176421,1.494665
6,2024-07-24 11:05,SIE,STK,1.0,0.082425,181.640000,172.812302,-4.859997,,"Stock(conId=14217, symbol='SIE', right='0', pr...",...,,False,,172.812302,-8.827698,EUR,0.0,DU3180649,172.810000,1.000000
7,2024-07-24 11:05,VOW,STK,-176.0,-9.306355,141.129400,110.860001,21.447976,,"Stock(conId=14232, symbol='VOW', right='0', pr...",...,,False,,-19511.360106,-30.269399,EUR,0.0,DU3180649,-19511.360000,1.000000
8,2024-07-24 11:21,AAPL,STK,37.0,3.647607,180.260541,224.490005,24.536410,,"Stock(conId=265598, symbol='AAPL', right='0', ...",...,,False,,8306.130203,44.229465,USD,0.0,DU3180649,7648.001627,1.083585
9,2024-07-24 11:21,CNR,STK,-699.0,-36.921605,174.287153,165.533997,5.022261,,"Stock(conId=4458894, symbol='CNR', right='0', ...",...,,False,,-115708.263623,-8.753157,CAD,0.0,DU3180649,-77414.178845,1.494665
